task name: CustomMappingSaveAndLoad
task description: Implement a custom mapping transformation in ML.NET and save the transformed data to an output file.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Transforms;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class InputData
{
    public float Age { get; set; }
}

In [ ]:
var samples = new List<InputData>
{
    new InputData { Age = 26 },
    new InputData { Age = 35 },
    new InputData { Age = 34 },
    new InputData { Age = 28 },
};
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
[CustomMappingFactoryAttribute("IsUnderThirty")]
private class IsUnderThirtyCustomAction : CustomMappingFactory<InputData,
    CustomMappingOutput>
{
    // We define the custom mapping between input and output rows that will
    // be applied by the transformation.
    public static void CustomAction(InputData input, CustomMappingOutput
        output) => output.IsUnderThirty = input.Age < 30;

    public override Action<InputData, CustomMappingOutput> GetMapping()
        => CustomAction;
}

// Defines only the column to be generated by the custom mapping
// transformation in addition to the columns already present.
private class CustomMappingOutput
{
    public bool IsUnderThirty { get; set; }
}

In [ ]:
var pipeline = mlContext.Transforms.CustomMapping(new
    IsUnderThirtyCustomAction().GetMapping(), contractName:
    "IsUnderThirty");

In [ ]:
var transformer = pipeline.Fit(data);

In [ ]:
mlContext.ComponentCatalog.RegisterAssembly(typeof(IsUnderThirtyCustomAction).Assembly);

In [ ]:
mlContext.Model.Save(transformer, data.Schema, "customTransform.zip");

In [ ]:
var transformedData = transformer.Transform(data);

In [ ]:
public class TransformedData : InputData
{
    public bool IsUnderThirty { get; set; }
}

In [ ]:
var dataEnumerable = mlContext.Data.CreateEnumerable<TransformedData>(
    transformedData, reuseRowObject: true);

Console.WriteLine("Age\tIsUnderThirty");
foreach (var row in dataEnumerable)
    Console.WriteLine($"{row.Age}\t {row.IsUnderThirty}");